In [3]:
import os, time, pickle

import numpy as np
np.random.seed(42)

import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Data Attribute Information

# 1. Sample code number: id number 
# 2. Clump Thickness: 1 - 10 
# 3. Uniformity of Cell Size: 1 - 10 
# 4. Uniformity of Cell Shape: 1 - 10 
# 5. Marginal Adhesion: 1 - 10 
# 6. Single Epithelial Cell Size: 1 - 10 
# 7. Bare Nuclei: 1 - 10 
# 8. Bland Chromatin: 1 - 10 
# 9. Normal Nucleoli: 1 - 10 
# 10. Mitoses: 1 - 10 
# 11. Class: (2 for benign, 4 for malignant)

# Specifing header
header = ['sample_code','clump_tickness','uniformity_cell_size','uniformity_cell_shape',
          'marginal_adhesion','single_epithelial_cell_size','bare_nuclei','bland_chromatin',
          'normal_nucleoli','mitoses','target']

# Reading the data
folderName = '../data/' 
fileName   = 'breast-cancer-wisconsin.data'
model_file_name   = 'decisionTree_model.sav'

df = pd.read_csv(folderName+fileName,header=None,names=header)

# Printing some info about the data
print("[INFO]\nNrow: {}\nNcol: {}".format(df.shape[0],df.shape[1]))
df.head(2)

[INFO]
Nrow: 683
Ncol: 11


,sample_code,clump_tickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses,target
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2


In [5]:
# Setting the target column
target = df.loc[:,df.columns[-1]].values
# target = np.where(target==4,1,0)

# If necessary, dropping cols
cols = df.columns # List of columns in dataframe
drop = [cols[-1],cols[0]] # List of columns to drop
df.drop(labels=drop,axis=1,inplace=True)
df.head(3)

,clump_tickness,uniformity_cell_size,uniformity_cell_shape,marginal_adhesion,single_epithelial_cell_size,bare_nuclei,bland_chromatin,normal_nucleoli,mitoses
0,5,1,1,1,2,1,3,1,1
1,5,4,4,5,7,10,3,2,1
2,3,1,1,1,2,2,3,1,1


In [6]:
# Convert all columns to int or float to avoid errors
def conv2num(df):
    cols = df.columns
    for col in cols:
        try:
            df[str(col)] = df[str(col)].astype(float) # Remember to specify type according to data specification
        except Exception as e:
            print('Column \'{}\' was not converted. Error: \n'.format(col), e, '\n')
            pass
    return df

df = conv2num(df) # Converting columns to number to avoid errors

In [7]:
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn import metrics

## K-Fold Cross Validation

In [13]:
# Initialize the classifier
clf = SVC(C=1.0, kernel='linear', degree=3)

# ----------- CROSS VALIDATION ----------- #
# ---------------- K-Fold ---------------- #
accuracy_train=[]
precision_train=[]
recall_train=[]
f1_train=[]
auc_train=[]

accuracy_test=[]
precision_test=[]
recall_test=[]
f1_test=[]
auc_test=[]
cv = model_selection.KFold(n_splits=10,shuffle=False) # K-fold Cross Validation method
for train_index, test_index in cv.split(df.values):
    X_train, X_test = df.values[train_index,:], df.values[test_index,:]
    y_train, y_test = target[train_index],target[test_index]  
    
    # Fitting the data into the model
    clf.fit(X_train, y_train)
    
    # FOR TRAINING
    predicted_train = clf.predict(X_train)
    accuracy_train.append(metrics.accuracy_score(y_train, predicted_train))
    precision_train.append(metrics.precision_score(y_train, predicted_train,pos_label=4))
    recall_train.append(metrics.recall_score(y_train, predicted_train,pos_label=4))
    f1_train.append(metrics.f1_score(y_train, predicted_train,pos_label=4))
    fpr, tpr, thresholds = metrics.roc_curve(y_train, predicted_train,pos_label=4)
    auc_train.append(metrics.auc(fpr, tpr))
    
    # FOR TESTING
    predicted_test = clf.predict(X_test)
    accuracy_test.append(metrics.accuracy_score(y_test, predicted_test))
    precision_test.append(metrics.precision_score(y_test, predicted_test,pos_label=4))
    recall_test.append(metrics.recall_score(y_test, predicted_test,pos_label=4))
    f1_test.append(metrics.f1_score(y_test, predicted_test,pos_label=4))
    fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted_test,pos_label=4)
    auc_test.append(metrics.auc(fpr, tpr))

In [14]:
# Calculating the mean values for train and test scores
accuracy_train = np.array(accuracy_train).mean()
precision_train = np.array(precision_train).mean()
recall_train = np.array(recall_train).mean()
f1_train = np.array(f1_train).mean()
auc_train = np.array(auc_train).mean()

accuracy_test = np.array(accuracy_train).mean()
precision_test = np.array(precision_train).mean()
recall_test = np.array(recall_train).mean()
f1_test = np.array(f1_train).mean()
auc_test = np.array(auc_train).mean()

# Displaying the results
print('Accuracy Train = {}'.format(accuracy_train))
print('Precision Train = {}'.format(precision_train))
print('Recall Train = {}'.format(recall_train))
print('F1-score Train = {}'.format(f1_train))
print('AUC Train = {}'.format(auc_train))
print()
print('Accuracy Test = {}'.format(accuracy_test))
print('Precision Test = {}'.format(precision_test))
print('Recall Test = {}'.format(recall_test))
print('F1-score Test = {}'.format(f1_test))
print('AUC Test = {}'.format(auc_test))

Accuracy Train = 0.9729951537300389
Precision Train = 0.954772224838744
Recall Train = 0.9688667034885601
F1-score Train = 0.9617588490526143
AUC Train = 0.9720143335278048

Accuracy Test = 0.9729951537300389
Precision Test = 0.954772224838744
Recall Test = 0.9688667034885601
F1-score Test = 0.9617588490526143
AUC Test = 0.9720143335278048


## Hold-Out Cross Validation

In [15]:
# Initialize the classifier
clf = SVC(C=1.0, kernel='linear', degree=3)

accuracy_train=[]
precision_train=[]
recall_train=[]
f1_train=[]
auc_train=[]

accuracy_test=[]
precision_test=[]
recall_test=[]
f1_test=[]
auc_test=[]

# ----------- CROSS VALIDATION ----------- #
# -------------- Hold-Out -------------- # When I have too many data
X_train, X_test, y_train, y_test  = model_selection.train_test_split(df.values, 
                                                                     target, 
                                                                     test_size=0.5,
                                                                     random_state=None,
                                                                     shuffle=False)

# Fitting the data into the model
clf.fit(X_train, y_train)

# FOR TRAINING
predicted_train = clf.predict(X_train)
accuracy_train.append(metrics.accuracy_score(y_train, predicted_train))
precision_train.append(metrics.precision_score(y_train, predicted_train,pos_label=4))
recall_train.append(metrics.recall_score(y_train, predicted_train,pos_label=4))
f1_train.append(metrics.f1_score(y_train, predicted_train,pos_label=4))
fpr, tpr, thresholds = metrics.roc_curve(y_train, predicted_train,pos_label=4)
auc_train.append(metrics.auc(fpr, tpr))

# FOR TESTING
predicted_test = clf.predict(X_test)
accuracy_test.append(metrics.accuracy_score(y_test, predicted_test))
precision_test.append(metrics.precision_score(y_test, predicted_test,pos_label=4))
recall_test.append(metrics.recall_score(y_test, predicted_test,pos_label=4))
f1_test.append(metrics.f1_score(y_test, predicted_test,pos_label=4))
fpr, tpr, thresholds = metrics.roc_curve(y_test, predicted_test,pos_label=4)
auc_test.append(metrics.auc(fpr, tpr))

In [16]:
# Calculating the mean values for train and test scores
accuracy_train = np.array(accuracy_train).mean()
precision_train = np.array(precision_train).mean()
recall_train = np.array(recall_train).mean()
f1_train = np.array(f1_train).mean()
auc_train = np.array(auc_train).mean()

accuracy_test = np.array(accuracy_train).mean()
precision_test = np.array(precision_train).mean()
recall_test = np.array(recall_train).mean()
f1_test = np.array(f1_train).mean()
auc_test = np.array(auc_train).mean()

# Displaying the results
print('Accuracy Train = {}'.format(accuracy_train))
print('Precision Train = {}'.format(precision_train))
print('Recall Train = {}'.format(recall_train))
print('F1-score Train = {}'.format(f1_train))
print('AUC Train = {}'.format(auc_train))
print()
print('Accuracy Test = {}'.format(accuracy_test))
print('Precision Test = {}'.format(precision_test))
print('Recall Test = {}'.format(recall_test))
print('F1-score Test = {}'.format(f1_test))
print('AUC Test = {}'.format(auc_test))

Accuracy Train = 0.9648093841642229
Precision Train = 0.9451219512195121
Recall Train = 0.9810126582278481
F1-score Train = 0.9627329192546583
AUC Train = 0.9659161651794976

Accuracy Test = 0.9648093841642229
Precision Test = 0.9451219512195121
Recall Test = 0.9810126582278481
F1-score Test = 0.9627329192546583
AUC Test = 0.9659161651794976


## Leave-One-Out Cross Validation

In [17]:
# Initialize the classifier
clf = SVC(C=1.0, kernel='linear', degree=3)

# ----------- CROSS VALIDATION ----------- #
# ---------------- K-Fold ---------------- #
accuracy_train=[]
precision_train=[]
recall_train=[]
f1_train=[]
auc_train=[]

pred_test=[]
accuracy_test=[]
precision_test=[]
recall_test=[]
f1_test=[]
auc_test=[]
cv = model_selection.LeaveOneOut() # K-fold Cross Validation method
for train_index, test_index in cv.split(df.values):
    X_train, X_test = df.values[train_index,:], df.values[test_index,:]
    y_train, y_test = target[train_index],target[test_index]  
    
    # Fitting the data into the model
    clf.fit(X_train, y_train)
        
    # FOR TRAINING
    predicted_train = clf.predict(X_train)
    accuracy_train.append(metrics.accuracy_score(y_train, predicted_train))
    precision_train.append(metrics.precision_score(y_train, predicted_train,pos_label=4))
    recall_train.append(metrics.recall_score(y_train, predicted_train,pos_label=4))
    f1_train.append(metrics.f1_score(y_train, predicted_train,pos_label=4))
    fpr, tpr, thresholds = metrics.roc_curve(y_train, predicted_train,pos_label=4)
    auc_train.append(metrics.auc(fpr, tpr))

    # Storing testing
    pred_test.append([y_test, clf.predict(X_test)])

# Converting to array
pred_test = np.array(pred_test)
    
# FOR TESTING
accuracy_test = metrics.accuracy_score(pred_test[:,0,0], pred_test[:,1,0])
precision_test = metrics.precision_score(pred_test[:,0,0], pred_test[:,1,0],pos_label=4)
recall_test = metrics.recall_score(pred_test[:,0,0], pred_test[:,1,0],pos_label=4)
f1_test = metrics.f1_score(pred_test[:,0,0], pred_test[:,1,0],pos_label=4)
fpr, tpr, thresholds = metrics.roc_curve(pred_test[:,0,0], pred_test[:,1,0],pos_label=4)
auc_test = metrics.auc(fpr, tpr)

In [18]:
# Calculating the mean values for train and test scores
accuracy_train = np.array(accuracy_train).mean()
precision_train = np.array(precision_train).mean()
recall_train = np.array(recall_train).mean()
f1_train = np.array(f1_train).mean()
auc_train = np.array(auc_train).mean()

# Displaying the results
print('Accuracy Train = {}'.format(accuracy_train))
print('Precision Train = {}'.format(precision_train))
print('Recall Train = {}'.format(recall_train))
print('F1-score Train = {}'.format(f1_train))
print('AUC Train = {}'.format(auc_train))
print()
print('Accuracy Test = {}'.format(accuracy_test))
print('Precision Test = {}'.format(precision_test))
print('Recall Test = {}'.format(recall_test))
print('F1-score Test = {}'.format(f1_test))
print('AUC Test = {}'.format(auc_test))

Accuracy Train = 0.9736027444901953
Precision Train = 0.9546346065664855
Recall Train = 0.9706927644462712
F1-score Train = 0.9625960822796737
AUC Train = 0.9729309355670694

Accuracy Test = 0.9677891654465594
Precision Test = 0.9465020576131687
Recall Test = 0.9623430962343096
F1-score Test = 0.9543568464730291
AUC Test = 0.966531908477515
